In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio; pio.renderers.default='notebook'

In [ ]:
# data source:
# https://opendata.terrassa.cat/dataset/mapa-de-la-ciutat-amb-les-divisions-per-seccions-censals
geodf = gpd.read_file('../maps/pt_cens.shp').to_crs(epsg=4326) # else plotly may complain

# data source: 
# https://analisi.transparenciacatalunya.cat/Societat-benestar/Eleccions-al-Parlament-de-Catalunya-2021-Recompte-/ix2p-vyw4
vots = pd.read_csv('../data/Eleccions_al_Parlament_de_Catalunya_2021_-_Recompte_provisional_per_mesa.csv')
vots=vots.loc[vots['Codi municipi']==279] # discard all but Terrassa
print(vots.shape)
vots.head()

In [ ]:
# aggregate per section
df = vots.groupby(['Districte','Secció']).sum()
df = df[[x for x in df.columns if not x.startswith('%')]] # get rid of %, we might calculate them later
print(df.shape)
df.head()

In [ ]:
df.columns

In [ ]:
newcolnames = df.columns.tolist()[:10]+[x.split('(')[-1][:-1] for x in df.columns[10:]] # keep acronym per party
df.columns = newcolnames
df = df[[x for x in df.columns if not x.startswith('Codi')]] # drop nonsense aggregated data
df.head()

In [ ]:
partits = df.iloc[:,7:].columns
df['partit_max'] = df.iloc[:,7:].values.argmax(axis=1)
df['perc_abst'] = df['Abstenció'] * 100 / df['Cens electoral']
df.head()

In [ ]:
df.partit_max.unique() # just PSC, ERC amd JxCat

In [ ]:
for i, partit in enumerate(partits):
    df[f'perc_{partit}'] = 100*df[partit]/df['Vots vàlids']
    
df.head()

In [ ]:
# # we'll retake this later falta color per representar
colors = {
    'PSC':'#e10819',#ff', PSC # 0 indx in partits
    'ERC':'#ffe210',#ff', # ERC # 1
    'JxCat': '#20c0b2',#ff' # junts # 3
         }
dict_nom_index_partit = dict(
    zip(np.arange(partits.size), partits)
)
#df['partit_mes_votat'] = df.partit_max.map(colors)
df['partit_mes_votat'] = df.partit_max.map(dict_nom_index_partit)


In [ ]:
# we'll generate a json file due plotly failing to read geometry directly from geopandas
geodf.to_file("terrassa_cens.json", driver = "GeoJSON")
import json
with open("terrassa_cens.json") as geofile:
    j_file = json.load(geofile)
    
# add id feature (default name plotly will look for) to link it later with our geodf.index
for i,feature in enumerate(j_file["features"]):
    feature['id'] = str(i)
    
geodf.head(1)

In [ ]:
#geodf.dtypes
for col in ['DIST', 'SECC']:
    geodf[col]=geodf[col].astype(int)

In [ ]:
# merge geodata with elections
merged = pd.merge(
    geodf, 
    df.reset_index(), 
    how='left', # left join keeps our geodf index
    left_on=['DIST', 'SECC'],
    right_on=['Districte', 'Secció']
)
merged.head(2)

In [ ]:
fig = px.choropleth_mapbox(merged.loc[merged['Vots vàlids']>0], #.to_crs(epsg=4326),
                           geojson=j_file,
                           locations=merged.loc[merged['Vots vàlids']>0].index,
                           range_color=(0,100),
                           color=(100-merged.loc[merged['Vots vàlids']>0,"perc_abst"]).round(decimals=2),
                           color_continuous_scale="magma",
                           center={"lat": 41.56, "lon": 2.0},
                           mapbox_style='carto-positron',
                           hover_data=['DIST', 'SECC'],
                           labels={'color':'participació'},
                           opacity=0.5,
                           zoom=11)
fig.update_layout(
    coloraxis_colorbar=dict(
        title="% Participació",
    ),
)
fig.write_html("../outputs/particip_14F_terrassa.html")
fig.show()

In [ ]:
fig = px.choropleth_mapbox(merged, 
                           geojson=j_file,
                           locations=merged.index,
                           color=merged["partit_mes_votat"],
                           #color_discrete_sequence='identity',
                           color_discrete_map=colors,
                           center={"lat": 41.56, "lon": 2.0},
                           mapbox_style='carto-positron',
                           hover_data=['DIST', 'SECC'],
                           opacity=0.5,
                           zoom=11)
fig.update_layout(title='Partit més votat @ Terrassa el 14F per secció',
                  legend={'title':'partit més votat'}
)
#fig.write_html("Vots_terrassa_secció.html") # there's something better below
fig.show()

In [ ]:
# try to refine it
# get 1r, 2n, 3r, 4t, 5e 6e and add it to hover as a string to display it interactively:
# we'll write a function to get it with a single apply call

def get_most_voted_strings(row):
    idxes=np.argsort(row[partits].values)[::-1] # get index for most voted
    names = partits[idxes[:8]]
    altres = partits[idxes[8:]]
    abs_vots = row.loc[names].values
    altres_abs_vots = row.loc[altres].values.sum()
    percvots = row.loc[[f'perc_{x}' for x in names]].values
    altres_perc_vots = row.loc[[f'perc_{x}' for x in altres]].values.sum()
    percvots = np.concatenate([percvots, [altres_perc_vots]])
    abs_vots = np.concatenate([abs_vots, [altres_abs_vots]])
    out = []
    for i, name in enumerate(names.tolist()+['altres']):
        out += [f'{name}, {round(percvots[i],2)}% ({abs_vots[i]} vots)']
        
    return out

# example how the function work
merged.iloc[:2].apply(lambda x: get_most_voted_strings(x), axis=1, result_type='expand')

In [ ]:
tmp = merged.apply(lambda row: get_most_voted_strings(row), axis=1, result_type='expand')
tmp.columns = ['1r', '2n', '3r', '4t', '5è', '6è', '7è', '8è','altres']
merged = pd.concat([merged, tmp], axis=1)
merged.head()

In [ ]:
fig = px.choropleth_mapbox(merged.reset_index(), 
                           geojson=j_file,
                           locations='index',
                           color="partit_mes_votat",
                           color_discrete_map=colors,
                           center={"lat": 41.56, "lon": 2.0},
                           mapbox_style='carto-positron',
                           hover_data={
                               'partit_mes_votat':False,
                               'index':False,
                               'DIST':True,'SECC':True, '1r':True, 
                               '2n':True, '3r':True, '4t':True, '5è':True, '6è':True, 
                               '7è':True, '8è':True,'altres':True},
                           opacity=0.5,
                           zoom=11)
fig.update_layout(title='Partit més votat @ Terrassa el 14F per secció',
                 legend={'title':'partit més votat'}
)
fig.write_html("../outputs/Vots_terrassa_seccio.html")
fig.show()

In [ ]:
### slider per party
# we need to reshape dataframe with wide_to_long or melt
melted = pd.melt(
    merged.reset_index(),
    id_vars=['index', 'DIST', 'SECC', 'geometry'],
    value_vars=[f'perc_{x}' for x in partits[:10]],
    value_name='perc',
    var_name='partit'
)
melted.partit=melted.partit.str.split('_').str[-1]
melted.head()

In [ ]:
# later me, also get votes. perhaps pivoting would have worked better
melted['vots_abs'] = pd.melt(
    merged.reset_index(),
    id_vars=['index', 'DIST', 'SECC', 'geometry'],
    value_vars=[f'{x}' for x in partits[:10]],
    value_name='vots_abs',
    var_name='partit'
)['vots_abs']

In [ ]:
melted.head(1)

In [ ]:
print(melted.partit.unique())
melted.perc.max()

In [ ]:
# we need to get a color for each of these and play with alpha value? # some colors might be trash
# just use a continuous colorscale instead

fig = px.choropleth_mapbox(melted.round(2), 
                           geojson=j_file,
                           locations='index',
                           range_color=(0,50),
                           color='perc',
                           color_continuous_scale="magma",
                           center={"lat": 41.56, "lon": 2.0},
                           mapbox_style='carto-positron',
                           #hover_data=['DIST', 'SECC'],
                           labels={'perc':'% vots/secció',
                                   'vots_abs': '# vots/secció'
                           },
                           animation_frame='partit',
                           opacity=0.5,
                           hover_data={
                               'partit':True,
                               'index':False,
                               'DIST':True,'SECC':True,
                               'perc':True, 'vots_abs':True # formatting petardeja
                           },
                           zoom=11)
fig.update_layout(
    coloraxis_colorbar=dict(
        title="% vots",
    ),
)
# not really accessible
# thanks god the community: https://community.plotly.com/t/how-to-slow-down-animation-in-plotly-express/31309/4
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 5000
fig.write_html("../outputs/perc_vots_partits.html")
fig.show()

In [ ]:
# meh not really clear. Try plottin diff from mean instead
# retake original df
perc_global = pd.DataFrame(
    100* df[partits[:10]].sum()/df['Vots vàlids'].sum()
).reset_index()
perc_global.columns = ['partit', 'perc_global']
melted = melted.merge(perc_global, how='left',
            on='partit')
melted['relative_perc'] = melted.perc-melted.perc_global
melted.relative_perc.describe()

In [ ]:
melted.head()

In [ ]:
fig = px.choropleth_mapbox(melted, 
                           geojson=j_file,
                           locations='index',
                           #range_color=(-30,30), # to get fair transitions (in magnitude)
                           color_continuous_midpoint=0, # more vivid
                           color=melted['relative_perc'].round(2),
                           color_continuous_scale="balance",
                           center={"lat": 41.56, "lon": 2.0},
                           mapbox_style='carto-positron',
                           hover_data=['DIST', 'SECC'],
                           labels={'color':'vs global ciutat'},
                           animation_frame='partit',
                           opacity=0.5,
                           zoom=11)
fig.update_layout(
    title='diferència respecte al promig de Terrassa',
    coloraxis_colorbar=dict(
        title="% vots",
    ),
)
# not really accessible
# thanks god the community: https://community.plotly.com/t/how-to-slow-down-animation-in-plotly-express/31309/4
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 5000
fig.write_html("../outputs/perc_vots_relatiu.html")
fig.show()

In [ ]:
# vale, ara experimentarem amb el datawrapper que mola bastant més el hover
# com aquí_ https://www.diaridegirona.cat/eleccions/catalanes/2021/02/14/mapa-interactiu-resultats-eleccions-catalanes/1089170.html

# l'api en python pseeee
# TBD